# Task 1: Video Action Recognition using Spatio-Temporal Modeling

In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install opencv-python tqdm matplotlib ffmpeg-python decord

In [ ]:
import os
import random
import torchvision
from glob import glob
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T
import torchvision.io as io
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import argparse
import os
from torch import nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import cv2
from utils import save_checkpoint, plot_metrics, visualize_activation_on_frames
from dataset import VideoFolderDataset


## Dataset

In [8]:

def uniform_sample_indices(num_frames, clip_len):
    if num_frames <= clip_len:
        # pad by repeating last frame
        repeats = list(range(num_frames)) + [num_frames-1]*(clip_len - num_frames)
        return repeats[:clip_len]
    interval = num_frames / float(clip_len)
    return [int(i * interval) for i in range(clip_len)]

class VideoFolderDataset(Dataset):
    """
    Expects dataset_root/class_name/*.mp4
    Returns: tensor (C,T,H,W), label
    """
    def __init__(self, root_dir, clip_len=16, transforms=None, extensions=('.mp4','.avi','.mov')):
        self.root_dir = root_dir
        self.clip_len = clip_len
        self.extensions = extensions
        self.transforms = transforms
        self.samples = []
        cls_dirs = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir,d))])
        self.class_to_idx = {c:i for i,c in enumerate(cls_dirs)}
        for c in cls_dirs:
            p = os.path.join(root_dir, c)
            for ext in extensions:
                for f in glob(os.path.join(p, f"**/*{ext}"), recursive=True):
                    self.samples.append((f, self.class_to_idx[c]))
        if len(self.samples)==0:
            raise RuntimeError(f"No videos found in {root_dir} with extensions {extensions}")

    def __len__(self):
        return len(self.samples)

    def read_video_rgb(self, path):
        # returns (T, H, W, C) numpy
        video, _, info = io.read_video(path, pts_unit='sec')  # (T, H, W, C)
        if isinstance(video, torch.Tensor):
            video = video.numpy()
        return video  # dtype uint8

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        video = self.read_video_rgb(path)  # (T,H,W,C) uint8
        num_frames = video.shape[0]
        indices = uniform_sample_indices(num_frames, self.clip_len)
        frames = video[indices]  # (clip_len,H,W,C)
        # convert to PIL per-frame and apply transforms if provided
        pil_frames = [Image.fromarray(fr) for fr in frames]
        if self.transforms:
            # apply same transform to each frame
            pil_frames = [self.transforms(f) for f in pil_frames]  # each -> tensor C,H,W
        else:
            # default convert to tensor and permute
            pil_frames = [T.ToTensor()(f) for f in pil_frames]

        # stack to shape (T, C, H, W) then permute to (C, T, H, W)
        frame_tensors = torch.stack(pil_frames)  # (T, C, H, W)
        frame_tensors = frame_tensors.permute(1, 0, 2, 3).contiguous()  # (C, T, H, W)
        return frame_tensors.float(), label


## Utility Functions

In [ ]:
def save_checkpoint(state, is_best, out_dir, name='checkpoint.pth'):
    os.makedirs(out_dir, exist_ok=True)
    path = os.path.join(out_dir, name)
    torch.save(state, path)
    if is_best:
        best_path = os.path.join(out_dir, 'model_best.pth')
        torch.save(state, best_path)

def plot_metrics(train_losses, val_losses, train_accs, val_accs, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    plt.figure(figsize=(8,4))
    plt.plot(train_losses, label='train_loss')
    plt.plot(val_losses, label='val_loss')
    plt.legend()
    plt.title('Loss')
    plt.savefig(os.path.join(out_dir, 'loss_curve.png'))
    plt.close()

    plt.figure(figsize=(8,4))
    plt.plot(train_accs, label='train_acc')
    plt.plot(val_accs, label='val_acc')
    plt.legend()
    plt.title('Top-1 Accuracy')
    plt.savefig(os.path.join(out_dir, 'acc_curve.png'))
    plt.close()

def visualize_activation_on_frames(frames_tensor, activation_map, out_path_prefix):
    """
    frames_tensor: (T, C, H, W) in [0,1] or [0,255]
    activation_map: (T, H, W) or (H, W) that aligns with frames
    Saves overlay images for each frame (simple grayscale heatmap overlay).
    """
    import matplotlib.pyplot as plt
    from matplotlib import cm
    os.makedirs(os.path.dirname(out_path_prefix), exist_ok=True)
    T = frames_tensor.shape[0]
    for t in range(T):
        frame = frames_tensor[t].permute(1,2,0).cpu().numpy()  # H,W,C
        if frame.max()>2:
            frame = frame/255.0
        act = activation_map[t] if activation_map.ndim==3 or activation_map.shape[0]==T else activation_map
        act = act if act.shape==(frame.shape[0], frame.shape[1]) else \
              np.array(Image.fromarray(act).resize((frame.shape[1], frame.shape[0])))
        plt.figure(figsize=(4,4))
        plt.imshow(frame)
        plt.imshow(act, cmap='jet', alpha=0.4)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f"{out_path_prefix}_frame{t:02d}.png")
        plt.close()


## Training

In [ ]:
def get_transforms(side_length=112):
    return T.Compose([
        T.Resize((side_length, side_length)),
        T.CenterCrop(side_length),
        T.ToTensor(),
        T.Normalize(
            mean=[0.43216, 0.394666, 0.37645],
            std=[0.22803, 0.22145, 0.216989]
        ),
    ])

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for x, y in tqdm(loader, desc='Train', leave=False):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        correct += (out.argmax(1) == y).sum().item()
        total += x.size(0)

    return running_loss / total, correct / total

@torch.no_grad()
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for x, y in tqdm(loader, desc='Val', leave=False):
        x, y = x.to(device), y.to(device)

        out = model(x)
        loss = criterion(out, y)

        running_loss += loss.item() * x.size(0)
        correct += (out.argmax(1) == y).sum().item()
        total += x.size(0)

    return running_loss / total, correct / total


In [ ]:
# === CONFIG ===
data_root   = "/workspace/KineticsMini/train"   # CHANGE THIS
out_dir     = "outputs_task1"
clip_len    = 16
batch_size  = 8
epochs      = 10
lr          = 1e-4
side_len    = 112
num_workers = 4
device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)


In [ ]:
transforms = get_transforms(side_len)

dataset = VideoFolderDataset(
    data_root,
    clip_len=clip_len,
    transforms=transforms
)

num_classes = len(dataset.class_to_idx)
print("Classes:", dataset.class_to_idx)

# Split
n_val = int(0.2 * len(dataset))
n_train = len(dataset) - n_val
train_set, val_set = random_split(dataset, [n_train, n_val])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader   = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [ ]:
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

train_losses, val_losses = [], []
train_accs, val_accs = [], []
best_acc = 0.0


In [ ]:
for epoch in range(1, epochs + 1):
    print(f"Epoch {epoch}/{epochs}")

    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    scheduler.step()

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    is_best = val_acc > best_acc
    best_acc = max(best_acc, val_acc)

    save_checkpoint(
        {
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc': best_acc,
            'class_to_idx': dataset.class_to_idx
        },
        is_best,
        out_dir,
        name=f"checkpoint_epoch{epoch}.pth"
    )

    plot_metrics(train_losses, val_losses, train_accs, val_accs, out_dir)

    print(f"Train Loss {train_loss:.4f} Acc {train_acc:.4f}")
    print(f"Val   Loss {val_loss:.4f} Acc {val_acc:.4f}  BEST {best_acc:.4f}")


In [ ]:
model.eval()

sample_x, _ = dataset[0]
sample_x = sample_x.unsqueeze(0).to(device)

activations = {}

def hook_fn(module, inp, outp):
    activations['feat'] = outp.detach().cpu()

handle = model.layer4[0].conv1.register_forward_hook(hook_fn)
_ = model(sample_x)
handle.remove()

feat = activations['feat']  # (1, C, T', H', W')
feat_mean = feat.squeeze(0).mean(0).numpy()  # (T', H', W')

up_acts = []
T_feat = feat_mean.shape[0]

for t in range(T_feat):
    a = feat_mean[t]
    resized = cv2.resize(a, (sample_x.shape[4], sample_x.shape[3]))
    up_acts.append(resized)

frames = sample_x.squeeze(0).permute(1,0,2,3).cpu()
visualize_activation_on_frames(frames, np.stack(up_acts), os.path.join(out_dir, "activation"))

print("Activation visualizations saved.")
